<a href="https://colab.research.google.com/github/mongbleu/Project_session1/blob/main/ds-section2-sprint2/n223-evaluation-metrics-for-classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 2 / NOTE 4*

# 📝 Assignment
---

# 모델선택(Model Selection)

### 1) 캐글 대회를 이어서 진행합니다. RandomizedSearchCV 를 사용하여 하이퍼파라미터 튜닝을 진행합니다.

- [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)를 사용하세요.
- 분류문제에서 맞는 [scoring parameter](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values) metric을 사용하세요.
- [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) 사용을 권합니다.
- RandomizedSearchCV 를 사용해서 하이퍼파라미터 튜닝을 진행하고 최고 성능을 보이는 모델로 예측을 진행한 후 캐글에 제출합니다.
- **(Urclass Quiz) 캐글 Leaderboard에서 개선된 본인 Score를 과제 제출폼에 제출하세요.**

In [2]:
import pandas as pd
target = 'vacc_h1n1_f'

train = pd.merge(pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv'), 
                 pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')[target], left_index=True, right_index=True)
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/test.csv')
sample_submission = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/submission.csv')

In [3]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, train_size=0.8, test_size=0.2, stratify=train[target],random_state=2)
train.shape, val.shape, test.shape

((33723, 39), (8431, 39), (28104, 38))

In [4]:
def engineer (df) :
  #cardinality가 30 이상이면 제거
  labels = df.columns
  columns = df.select_dtypes(include=['number','object'])
  cols_del = [col for col in columns if (len(df[col].unique())>=30)]
  df.drop(columns=cols_del, inplace=True)

  #behav 합치기
  new_behav = [col for col in labels if 'behavioral' in col] #list.contains 없음
  df['behavioral'] = df[new_behav].sum(axis=1) #행 방향으로 더하기

  #doctor_recc_seasonal,employment 제거 (다중공선성, 높은 결측률)
  cols_miss = [col for col in labels if (df[col].isnull().sum()/len(df.index)>=0.5)]
  df.drop(columns=cols_miss,inplace=True)
  
  return df

In [8]:
train=engineer(train)
val= engineer(val)
test = engineer(test) 
train.shape, val.shape, test.shape

((33723, 37), (8431, 37), (28104, 36))

In [9]:
labels = [col for col in train.columns if 'behavioral_' in col]
train.drop(columns=labels,inplace=True)
val.drop(columns=labels,inplace=True)
test.drop(columns=labels,inplace=True)

In [10]:
train.drop(columns='doctor_recc_seasonal',inplace=True)
test.drop(columns='doctor_recc_seasonal',inplace=True)
val.drop(columns='doctor_recc_seasonal',inplace=True)

In [11]:
X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
X_test = test

In [12]:
!pip install category_encoders

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [14]:
pipe = make_pipeline(
    TargetEncoder(min_samples_leaf=1, smoothing=1000),
    SimpleImputer(),
    RandomForestClassifier(max_depth=15, n_estimators=100, n_jobs=-1, random_state=2)
)

SyntaxError: ignored

In [14]:
#단일 하이퍼파리미터의 스코어를 보기 위한 함수 : validation curve
#근데 왜 array의 행이 3개일까?
from sklearn.model_selection import validation_curve
depth = range(1, 30, 2)
ts, vs = validation_curve(
    pipe, X_train, y_train
    , param_name='randomforestclassifier__max_depth'
    , param_range=depth, scoring='roc_auc'
    , cv=3
    , n_jobs=-1
)

KeyboardInterrupt: ignored

In [ ]:
ts

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
ts_mean = np.mean(ts,axis=1)
vs_mean = np.mean(vs,axis=1)

In [ ]:
fig, ax = plt.subplots()
#훈련데이터셋
ax.plot(depth, ts_mean, label='training_f1')
ax.plot(depth, vs_mean, label='validation_f1')
ax.set(title='Validation curve', xlabel='max_depth',ylabel='f1_score')
ax.legend() #legend 나오게
#적합한 max_depth는 5이다

In [15]:
#적합한 하이퍼파라미터의 조합 찾기
pipe = make_pipeline(
    TargetEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=2, n_jobs=-1, oob_score=True)
)

In [69]:
from scipy.stats import randint

In [70]:
hyper_params = {'targetencoder__min_samples_leaf': [5,10,15,20],
                'targetencoder__smoothing' : [2.,20.,50.,60.,100.,500.,1000.],
                'simpleimputer__strategy' : ['mean','median','most_frequent'],
                'randomforestclassifier__max_depth' : range(1,30,2),
                'randomforestclassifier__n_estimators' : randint(100,1000),
                'randomforestclassifier__max_features' : ['auto','sqrt','log2'],
                'randomforestclassifier__min_samples_leaf' : range(10,1001,10)
                }
#왜 numpy.arange를 쓰면 안될까??

In [71]:
from sklearn.model_selection import RandomizedSearchCV

In [72]:
from scipy.stats import randint
print(randint(50,500))

In [73]:
clf = RandomizedSearchCV(
    pipe,
    param_distributions=hyper_params,
    n_iter=100,
    scoring='f1',
    n_jobs=-1,
    refit=True,
    cv=3,
    random_state=2
)

In [74]:
!pip install -q "tqdm>=4.36.1"

In [75]:
!pip install -U tensorflow-addons

In [76]:
import tensorflow as tf
import tensorflow_addons as tfa

In [77]:
import tqdm

In [78]:
import sys
from IPython.lib import deepreload

stdout = sys.stdout
sys.stdout = open('junk','w')
deepreload.reload(tqdm)
sys.stdout = stdout

tqdm.__version__

'4.63.0'

In [79]:
tqdm_callback = tfa.callbacks.TQDMProgressBar()

In [80]:
clf.fit(X_train,y_train)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('targetencoder', TargetEncoder()),
                                             ('simpleimputer', SimpleImputer()),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(n_jobs=-1,
                                                                     random_state=2))]),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'randomforestclassifier__max_depth': range(1, 30, 2),
                                        'randomforestclassifier__max_features': ['auto',
                                                                                 'sqrt',
                                                                                 'log2'],
                                        'randomforestclassifier__min_samples_leaf': range(10, 1001, 10),
                                        'randomforestclassifier

In [82]:
clf.best_params_

{'randomforestclassifier__max_depth': 13,
 'randomforestclassifier__max_features': 'sqrt',
 'randomforestclassifier__min_samples_leaf': 30,
 'randomforestclassifier__n_estimators': 164,
 'simpleimputer__strategy': 'median',
 'targetencoder__min_samples_leaf': 10,
 'targetencoder__smoothing': 60.0}

In [85]:
y_pred = clf.predict(X_val)

In [88]:
from sklearn.metrics import roc_curve, roc_auc_score, classification_report

In [87]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.93      0.89      6416
           1       0.68      0.46      0.55      2015

    accuracy                           0.82      8431
   macro avg       0.76      0.70      0.72      8431
weighted avg       0.81      0.82      0.81      8431



In [96]:
y_pred_proba = clf.predict_proba(X_val)[:,1]

In [104]:
roc_auc_score(y_val, y_pred_proba, average='weighted')

0.8249023842379165

In [97]:
fpr, tpr, thresholds  = roc_curve(y_val, y_pred_proba)

In [100]:
optimal_idx = np.argmax(tpr-fpr)
optimal_thres = thresholds[optimal_idx]
optimal_thres

0.2799864196305923

In [103]:
y_pred_optimal = (y_pred_proba>= optimal_thres)

In [107]:
print(classification_report(y_val, y_pred_optimal))

              precision    recall  f1-score   support

           0       0.89      0.82      0.85      6416
           1       0.54      0.68      0.60      2015

    accuracy                           0.79      8431
   macro avg       0.72      0.75      0.73      8431
weighted avg       0.81      0.79      0.79      8431



In [114]:
y_test_pred = clf.predict(X_test)

In [111]:
y_test_pred = y_test_pred>=optimal_thres

In [115]:
sample_submission[target] = y_test_pred

In [117]:
sample_submission.to_csv('submission_5',index=False)

In [ ]:
#왜..성능이 떨어졌을까...

## 🔥 도전과제(Github - Discussion)


### 2) [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) 를 사용하여 하이퍼파라미터 튜닝을 진행합니다.
- 모델 성능을 높이기 위해 가능한 시도를 다 해보세요.
- 모델 성능 개선에 가장 큰 영향을 준 특성공학이나 하이퍼파라미터 튜닝에 대해서 왜 성능 개선에 큰 영향을 주었는지 설명해 보시고 서로의 결과에 대해 공유하고 토론해 보세요. 



In [16]:
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
import numpy as np

In [17]:
hyper_params = {'targetencoder__min_samples_leaf': [5,10,15,20],
                'targetencoder__smoothing' : [2.,20.,50.,60.,100.,500.,1000.],
                'simpleimputer__strategy' : ['mean','median','most_frequent'],
                'randomforestclassifier__criterion' : ['gini','entropy'],
                'randomforestclassifier__max_depth' : range(1,30,2),
                'randomforestclassifier__n_estimators' : np.arange(100,1000,10),
                'randomforestclassifier__max_features' : ['auto','sqrt','log2'],
                'randomforestclassifier__min_samples_leaf' : range(10,1001,10),
                'randomforest__min_impurity_decrease': [None,0.1,0.2,0.35,0.55, 0.65,0.9]
                }

In [18]:
gsc = GridSearchCV(
    pipe,
    param_grid=hyper_params,
    scoring='roc_auc',
    n_jobs=-1,
    refit=True,
    cv=3,
    return_train_score=True
)

In [ ]:
gsc.fit(X_train, y_train)
#램용량부족문제....